In [1]:
import sys
sys.path.append("../src/")
%load_ext autoreload
%autoreload 2
import faiss  
import torch

In [2]:
index = faiss.IndexFlatIP(100)   # build the index, d=size of vectors 

In [3]:
import numpy as np

In [4]:
xb = np.random.rand(100,100).astype(np.float32)

In [5]:
xb.dtype

dtype('float32')

In [6]:
# here we assume xb contains a n-by-d numpy matrix of type float32
index.add(xb)                  # add vectors to the index
print(index.ntotal)

100


In [7]:
xq = np.random.rand(2,100).astype(np.float32)

In [8]:
D, I = index.search(xq, 1)

In [9]:
D

array([[28.680365],
       [28.900494]], dtype=float32)

In [10]:
I

array([[ 7],
       [83]])

In [11]:
xq

array([[5.54404408e-02, 9.70241845e-01, 5.42561829e-01, 4.98816460e-01,
        3.60423803e-01, 6.46621704e-01, 7.44715273e-01, 4.82015073e-01,
        7.64980376e-01, 2.03234881e-01, 4.51820880e-01, 8.77226517e-02,
        9.63550270e-01, 3.22889358e-01, 7.31347919e-01, 6.65698588e-01,
        2.64263183e-01, 2.38808230e-01, 6.82938933e-01, 9.18583393e-01,
        3.28900784e-01, 5.23584366e-01, 2.33002394e-01, 2.31518969e-02,
        8.76193762e-01, 2.00873077e-01, 9.34594035e-01, 7.16190636e-01,
        3.80562931e-01, 1.62701443e-01, 4.36069667e-01, 8.61753166e-01,
        5.05655468e-01, 8.26638818e-01, 7.57649481e-01, 1.85768127e-01,
        5.20699024e-01, 5.02426028e-02, 2.03172907e-01, 5.80699623e-01,
        4.00809556e-01, 6.54295504e-01, 5.51003456e-01, 4.18148875e-01,
        2.29290023e-01, 5.40377378e-01, 9.36542094e-01, 4.37755466e-01,
        6.62251413e-01, 5.49782395e-01, 8.06210995e-01, 4.61299568e-02,
        1.85929030e-01, 8.21717143e-01, 9.58036184e-01, 9.302231

In [12]:
xb[32]

array([0.36709774, 0.01868419, 0.7844161 , 0.13255318, 0.01773141,
       0.46975434, 0.6031421 , 0.8325629 , 0.03444909, 0.6948054 ,
       0.03885154, 0.33044648, 0.14642146, 0.52995616, 0.11065452,
       0.5043714 , 0.11057313, 0.19892928, 0.83174425, 0.31607446,
       0.8923153 , 0.53857535, 0.72438264, 0.76872605, 0.19023292,
       0.1933642 , 0.39589378, 0.05573777, 0.8583997 , 0.84888303,
       0.20365712, 0.26207957, 0.8564358 , 0.12472252, 0.45748293,
       0.59283   , 0.93222123, 0.41048843, 0.04060174, 0.8758207 ,
       0.1422287 , 0.907882  , 0.7169842 , 0.47282323, 0.4419565 ,
       0.07808255, 0.59086484, 0.01656216, 0.28013968, 0.6866983 ,
       0.17540969, 0.55518633, 0.23562117, 0.30155936, 0.68702507,
       0.39415923, 0.5698617 , 0.47218543, 0.9824452 , 0.9529985 ,
       0.54422194, 0.4511032 , 0.44910878, 0.14000134, 0.46984   ,
       0.8565914 , 0.7001015 , 0.6423666 , 0.93737584, 0.8731875 ,
       0.9646698 , 0.90118146, 0.39553335, 0.0117071 , 0.66705

In [122]:
# Load model
device = torch.device("cpu")
with open("../checkpoints/chkpt-1-3000-model.pt", 'rb') as f:
    model = torch.load(f, map_location=device)

In [39]:
from torch.utils.data import DataLoader
import data
import glob

val_data_files = glob.glob("../data/val/*.pkl")
valid_dataset = data.HierLMDataset(val_data_files, seq_len=64, device=device,
                                  provenance=True)
valid_data_loader = DataLoader(valid_dataset, batch_size=64,
                                    num_workers=0)

In [40]:
for batch_index, batch in enumerate(valid_data_loader):
    break

In [48]:
batch[1][0][0][0], batch[1][0][1][0] # (text is at 1, seq, [tag/text], batch)

('V', 'demonstrated')

In [59]:
batch_data = batch[0]
batch_text = batch[1]

In [58]:
for i in range(64):
    print(batch[1][i][1][0])

demonstrated
the helicopter
its early battlefield capabilities
in the korean war
began
the united states army
experimenting with expanded roles and missions for them
after the helicopter demonstrated its early battlefield capabilities in the korean war
experimenting
the united states army
with expanded roles and missions for them
expanded
roles and
culminated
this
with the formation of the tactical mobility requirements board , which strongly recommended the adaptation of helicopters in a variety of roles and missions
in 1962
recommended
the tactical mobility requirements board
the adaptation of helicopters in a variety of roles and missions
strongly
was
activated
the 11th air assault division
in 1963 to test a number of the board ' s concepts and recommendations about air mobility
test
the 11th air assault division
a number of the board ' s concepts and recommendations about air mobility
assigned
the 155mm howitzers
to army divisions
then as medium support artillery
were
the 155mm how

In [62]:
batch_data.shape

torch.Size([64, 65, 768])

In [63]:
grab_all_vecs = batch_data[:,1:].reshape((-1, batch_data.shape[2]))

In [64]:
grab_all_vecs.shape

torch.Size([4096, 768])

In [65]:
# Create index
index = faiss.IndexFlatIP(batch_data.shape[2])   # build the index, d=size of vectors 

In [66]:
index.add(grab_all_vecs.numpy().astype(np.float32))

In [67]:
output = model(batch_data[0,0].view(1,1,-1))

In [68]:
output.shape

torch.Size([1, 1, 768])

In [69]:
D, I = index.search(output[0,0].detach().numpy().astype(np.float32).reshape(1,-1), 1)

In [70]:
D

array([[3.2349136]], dtype=float32)

In [71]:
I

array([[879]])

In [75]:
seq = I[0][0] % 64
bb = I[0][0] // 64
(batch_text[seq][0][bb], batch_text[seq][1][bb])

('V', 'entered')

In [82]:
model.eval()
pass

In [124]:
input_seq = batch_data[0,0].view(1,1,-1)
history = []
for i in range(25):
    output = model(input_seq)
    new_tok = output[i,0].detach().numpy().astype(np.float32).reshape(1,-1)
    D, I = index.search(new_tok, 5)
    choose = 0
    new_input_idx = I[0][choose]
    while history and new_input_idx in history:
        choose += 1
        if choose >= I.shape[1]:
            new_input_idx = np.random.choice(64*64)
        else:
            new_input_idx = I[0][choose]
    history.append(new_input_idx)
    # print(new_input_idx)
    seq = new_input_idx % 64
    bb = new_input_idx // 64
    print((batch_text[seq][0][bb], batch_text[seq][1][bb]))
    new_input = grab_all_vecs[new_input_idx]
    new_input = new_input.view(1,1,-1)
    input_seq = torch.cat([input_seq, new_input], axis=0)
    # print(input_seq.shape)

('V', 'rejected')
('ARG0', 'quist')
('V', 'has')
('ARGM', 'purposely')
('V', 'made')
('ARG0', 'lawmakers')
('ARGM', 'legally')
('V', 'own')
('V', 'owning')
('ARG2', 'from owning a firearm')
('ARG1', 'a firearm')
('ARG1', 'a gun')
('ARG1', 'a lasting political impact on events in india')
('V', 'had')
('ARG0', 'the great famine')
('ARG1', 'the great famine')
('V', 'occurred')
('ARG1', 'the famine')
('ARG1', 'famine')
('V', 'affected')
('V', 'affected')
('ARG0', 'it')
('ARGM', 'for a period of two years')
('V', 'covered')
('ARG1', 'famine')
